In [4]:
using LinearAlgebra
using SparseArrays

using IterativeSolvers
using Preconditioners

using BenchmarkTools

using Plots
using LaTeXStrings

In [ ]:
N = 0;
A = 0;
f = 0;
p = 0;

N_ = [10000];
t  = zeros(length(N_), 6);
for i = 1:length(N_)
    N = N_[i];
    
    ## Setup
    Np1 = N+1; Nm1 = N-1; Np2 = Np1*Np1; Nm2 = Nm1*Nm1; Nbnd = 4*N; h = 1/N; h2=h*h; 

    #..construct the 2D mesh (X) starting from the 1D mesh (x)  
    #..observe that we make in the 2D mesh the x coordinate increase from left to right and 
    #..the y coordinate increase from top to bottom (as expected)
    x = Vector(0:h:1); 
    y = Vector(0:h:1); 
    X = repeat(reshape(x, 1, :), length(y), 1);
    Y = repeat(y, 1, length(x));

    #..construct the mesh indicator matrix IG 
    #..this indicator matrix will allow to distinguish interior and boundary nodes 
    #..in this indicator matrix the boundary nodes are easy to identify 
    #..for interior nodes IG(i,j) = 0 and for the boundary nodes IG(i,j) = 1
    #..next construct the indicator vector IGvec by reshaping the indicator matrix IG  
    IG = ones(Np1,Np1); 
    IG[2:end-1,2:end-1] = zeros(Nm1,Nm1); 
    IGvec = sparse(reshape(IG,Np2,1));

    #..construct array with linear indices allowing to define interior and boundary nodes 
    #..interior: index array with all indices of the interior nodes 
    #..boundary: index array with indices of all the boundary nodes 
    L = LinearIndices(IGvec); 
    interior_cartesian = findall(x->x==0,IGvec);  interior = L[interior_cartesian]; 
    boundary_cartesian = findall(x->x>0,IGvec); boundary = L[boundary_cartesian]; 
    
    sourceterm(x,y) = - 2*x*(x-1) - 2*y*(y-1); 
    #..Evaluate fsource on each node of the grid (Xh)
    #..Observe the use of the dot syntax to evaluate fsource in all mesh nodes in X 
    F = sourceterm.(X,Y); 
    #..Reshape the F 2D array into an f vector 
    f = reshape(F,Np2);   
    
    e = ones(Np1);            #..same as in 1D.. 
    e_bnd = ones(Nbnd); #..used to handle the boundary nodes 
    
    A1    = sparse(Tridiagonal(-e[2:end], 2*e, -e[2:end]) / h2); 
    I_int = Diagonal(e); 
    I_bnd = Diagonal(e_bnd); 
    
    A     = kron(A1,I_int) + kron(I_int,A1);
    
    # Boundary conditions
    A[boundary,boundary] = I_bnd;
    A[boundary,interior] = zeros(Nbnd, Nm2); A[interior, boundary] = zeros(Nm2, Nbnd);
    f[boundary] = zeros(Nbnd); 
    
    ## Benchmark
    #bm = @benchmark A \ f
    #t[i, 1] = mean(bm).time;
    
    #p = DiagonalPreconditioner(A);
    #bm = @benchmark cg(A, f, Pl=p, log=true)
    #t[i, 2] = mean(bm).time;
    
    #p = AMGPreconditioner{RugeStuben}(A);
    #bm = @benchmark cg(A, f, Pl=p, log=true)
    #t[i, 3] = mean(bm).time;
    
    #p = AMGPreconditioner{SmoothedAggregation}(A);
    #bm = @benchmark cg(A, f, Pl=p, log=true)
    #t[i, 4] = mean(bm).time;
    
    #bm = @benchmark minres(A, f, log = true)
    #t[i, 5] = mean(bm).time;
    
    p  = CholeskyPreconditioner(A, 2);
    bm = @benchmark cg(A, f, Pl=p, log=true)
    t[i, 6] = mean(bm).time;
end

t

## Results

In [23]:
# Saved computation times [ns] as a function of N
N_ = [8, 16, 32, 64, 128];
t = [2.84771e5 38257.8 5.30779e6 7.20529e5 17114.0 14259.0;
     1.0926e6 4.10917e5 4.04907e7 5.5482e6 53432.6 82712.5;
     3.84079e6 3.38527e6 1.49527e8 8.27664e7 3.52783e5 7.29705e5;
     1.60575e7 1.70994e7 8.7514e9 1.23818e9 4.15758e6 6.15263e6;
     5.86411e7 2.2956e8 4.22862e10 2.91611e10 1.30279e8 8.15051e7];

p1 = plot(N_, t * 1e-6, yaxis = :log, shape = [:circle :rect :utriangle :utriangle :diamond :rect], xlabel = L"N", ylabel = "Time [ms]", legend = :bottomright, label = ["Backslash" "DiagonalPreconditioner" "RugeStuben" "SmoothedAggregation" "MINRES" "CholeskyPreconditioner"])
#savefig(p1, "preconditioner_results1.png")

# Troubleshooting
The Cholesky preconditioner needs the $A$ matrix to be symmetric and positive definite. The former is not the case because of interior nodes that are coupled to one or more boundary nodes.

In [1]:
A = [1 0 0 0 0; -1 2 -1 0 0; 0 -1 2 -1 0; 0 0 -1 2 -1; 0 0 0 0 1]

5×5 Matrix{Int64}:
  1   0   0   0   0
 -1   2  -1   0   0
  0  -1   2  -1   0
  0   0  -1   2  -1
  0   0   0   0   1

In [4]:
isposdef(A)

false

This can be resolved easily by modifying the matrix as shown below.

_This only works for boundary nodes with Dirichlet condition?_

In [5]:
A = [1 0 0 0 0; 0 2 -1 0 0; 0 -1 2 -1 0; 0 0 -1 2 0; 0 0 0 0 1]

5×5 Matrix{Int64}:
 1   0   0   0  0
 0   2  -1   0  0
 0  -1   2  -1  0
 0   0  -1   2  0
 0   0   0   0  1

In [6]:
isposdef(A)

true